# Neural Network Force-Free magnetic field extrapolation - NF2
<img src="https://github.com/RobertJaro/NF2/blob/main/images/logo.jpg?raw=true" width="150" height="150">

This notebook uses NF2 to download and extrapolate vector magnetograms from SDO/HMI. Active regions and dates can be selected below. This version uses a pre-trained simulation as meta-state to achieve faster results.


GitHub Page: https://github.com/RobertJaro/NF2


## Installation and Imports

In [ ]:
!pip install git+https://github.com/RobertJaro/NF2.git

In [ ]:
import glob
import os

import numpy as np
from astropy.nddata import block_reduce
from sunpy.map import Map

import torch
from nf2.train.trainer import NF2Trainer
from nf2.data.download import download_HARP

import drms

from datetime import datetime

from urllib import request

## Data download

If you are looking for a NOAA active region, you can use the function below to search for the corrsponding HARP number.

In [ ]:
# Find SHARP number for given NOAA
#find_HARP(start_time, noaa_nums, client)

In [ ]:
#@title Active Region
sharp_nr = 5011 #@param {type:"number"}
year = 2015 #@param {type:"integer"}
month = 1 #@param {type:"integer"}
day = 3 #@param {type:"integer"}
hour = 8 #@param {type:"integer"}
minute = 0 #@param {type:"number"}

date = datetime(year, month, day, hour, minute)

download_dir = 'AR_5011' #@param {type:"string"}

Downloading data requires an active registration at JSOC. http://jsoc.stanford.edu/ajax/register_email.html (free of charge)

In [ ]:
#@title Download Credentials
jsoc_email = 'robert.jarolim@uni-graz.at' #@param {type:"string"}

In [ ]:
client = drms.Client(email=jsoc_email, verbose=True)
download_HARP(sharp_nr, date, download_dir, client)

For training new extrapolations it is favorable to start from a pre-trained state. We use the weights of a different extrapolation to initialize our model. Although trained for a different active region, this model already fullfills the divergence- and force-free constrains, which enables a faster convergence. To start your training from scratch set `meta_path = None`.

In [ ]:
request.urlretrieve('http://kanzelhohe.uni-graz.at/nf2/meta.pt', 'meta.pt')

In [ ]:
#@title Meta state
meta_path = 'meta.pt'#@param {type:"raw"}


## Parameter Setup

In [ ]:
#@title Paths
base_path = None #@param {type:"string"}
base_path = 'ar_%d_%s' % (sharp_nr, date.isoformat('T')) if base_path is None else base_path
data_path = None #@param {type:"string"}
data_path = download_dir if data_path is None else download_path

In [ ]:
#@title Data Parameters
bin = 2 #@param {type:"integer"}
spatial_norm = 160  #@param {type:"integer"}
height = 320  #@param {type:"integer"}
b_norm = 2500  #@param {type:"number"}
d_slice =  None#@param {type:"raw"}

In [ ]:
#@title Model Parameters
dim = 256 #@param {type:"number"}

In [ ]:
#@title Training Parameters
lambda_div = 0.1 #@param {type:"number"}
lambda_ff = 0.1 #@param {type:"number"}
# use 300 for meta training and 800 when trained from scratch
epochs = 300 #@param {type:"number"}
epochs = int(epochs)
# decay not used for meta learning
decay_epochs = 500 #@param {type:"number"}
decay_epochs = int(decay_epochs)
batch_size = 1e4 #@param {type:"number"}
batch_size = int(batch_size)
n_samples_epoch = 1e6 #@param {type:"number"}
n_samples_epoch = int(n_samples_epoch)
log_interval = 10 #@param {type:"integer"}
validation_interval = 100 #@param {type:"integer"}
potential = True #@param {type:"boolean"}



In [ ]:
if isinstance(data_path, str):
    hmi_p = sorted(glob.glob(os.path.join(data_path, '*Bp.fits')))[0]  # x
    hmi_t = sorted(glob.glob(os.path.join(data_path, '*Bt.fits')))[0]  # y
    hmi_r = sorted(glob.glob(os.path.join(data_path, '*Br.fits')))[0]  # z
    err_p = sorted(glob.glob(os.path.join(data_path, '*Bp_err.fits')))[0]  # x
    err_t = sorted(glob.glob(os.path.join(data_path, '*Bt_err.fits')))[0]  # y
    err_r = sorted(glob.glob(os.path.join(data_path, '*Br_err.fits')))[0]  # z
else:
    hmi_p, err_p, hmi_r, err_r, hmi_t, err_t = data_path

hmi_cube = np.stack([Map(hmi_p).data, -Map(hmi_t).data, Map(hmi_r).data]).transpose()
error_cube = np.stack([Map(err_p).data, Map(err_t).data, Map(err_r).data]).transpose()
if d_slice is not None:
    hmi_cube = hmi_cube[d_slice[0]:d_slice[1], d_slice[2]:d_slice[3]]
    error_cube = error_cube[d_slice[0]:d_slice[1], d_slice[2]:d_slice[3]]

In [ ]:
# bin data
if bin > 1:
    hmi_cube = block_reduce(hmi_cube, (bin, bin, 1), np.mean)
    error_cube = block_reduce(error_cube, (bin, bin, 1), np.mean)

## Training

In [ ]:
trainer = NF2Trainer(base_path, hmi_cube, error_cube, height, spatial_norm, b_norm, dim,
                     potential_boundary=potential, lambda_div=lambda_div, lambda_ff=lambda_ff,
                     decay_epochs=decay_epochs, num_workers=4, meta_path=meta_path)

In [ ]:
trainer.train(epochs, batch_size, n_samples_epoch, log_interval, validation_interval)

## Evaluation

## Export

VTK files can be used to visualize the extrapolation results (e.g., Paraview). We use `tvtk` for converting the files.
The NF2 results require little storage (about 2 MB). It is faster to download the NF2 file and convert it on your local environment using the CPU resources.

The code below can be used to convert files in Colab.

In [ ]:
!pip install vtk==9.0.1
!pip install mayavi

In [ ]:
from nf2.evaluation.unpack import load_cube
from nf2.evaluation.vtk import save_vtk

In [ ]:
model_path = base_path + '/extrapolation_result.nf2'
vtk_path = base_path + '/extrapolation_result.vtk'

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
b = load_cube(model_path, device, progress=True)
save_vtk(b, vtk_path, 'B')